In [2]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, WindowOperations, DetrendOperations

import serial.tools.list_ports
import time

import numpy as np
import matplotlib.pyplot as plt

In [3]:
ports = serial.tools.list_ports.comports()

for port, desc, hwid in sorted(ports):
    print(f"{port}: {desc} [{hwid}]")

/dev/cu.Bluetooth-Incoming-Port: n/a [n/a]
/dev/cu.usbmodem11: Low Energy Dongle [USB VID:PID=2458:0001 SER=1 LOCATION=0-1]


In [4]:
serial_port = "/dev/cu.usbmodem11"

In [5]:
def get_alpha_beta(board, board_id):
    data = board.get_current_board_data(256)

    eeg_channels = BoardShim.get_eeg_channels(board_id)
    eeg_channel = eeg_channels[1]
    sampling_rate = BoardShim.get_sampling_rate(board_id)
    nfft = DataFilter.get_nearest_power_of_two(sampling_rate)
    DataFilter.detrend(data[eeg_channel], DetrendOperations.LINEAR.value)
    psd = DataFilter.get_psd_welch(data[eeg_channel], nfft, nfft // 2, sampling_rate, WindowOperations.HANNING.value)

    # calc band power
    alpha = DataFilter.get_band_power(psd, 7.0, 13.0)
    beta = DataFilter.get_band_power(psd, 14.0, 30.0)

    # print(alpha)
    return alpha

In [6]:
# setup
board_id = BoardIds.GANGLION_BOARD
params = BrainFlowInputParams()
params.serial_port = serial_port

# create a board
board = BoardShim(board_id, params)

if board.is_prepared():
        board.release_session()
board.prepare_session()
board.start_stream()

# retrive data
data = []
try:
    time.sleep(5)
    while len(data) < 50:
        time.sleep(1)
        datum = get_alpha_beta(board, board_id)
        data.append(datum)
finally:
    if board.is_prepared():
        board.release_session()

plt.plot(data)
plt.show()

[2023-11-05 17:21:19.418] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbmodem11",
    "timeout": 0
}
[2023-11-05 17:21:19.643] [board_logger] [info] use 15 as a timeout for device discovery and for callbacks
[2023-11-05 17:21:19.643] [board_logger] [info] mac address is not specified, try to find ganglion without it
[2023-11-05 17:21:25.688] [board_logger] [info] detected firmware version 2
[2023-11-05 17:21:25.688] [board_logger] [error] Make sure you provided correct port name and have permissions to open it(run with sudo/admin). And close all other apps using this port.
[2023-11-05 17:21:25.688] [board_logger] [error] failed to Open Ganglion Device 18


KeyboardInterrupt: 

In [7]:
import pickle
from huggingface_hub import hf_hub_download
from braindecode.models import EEGNetv4
import torch

# download the model from the hub:
path_kwargs = hf_hub_download(
    repo_id='PierreGtch/EEGNetv4',
    filename='EEGNetv4_BNCI2014004/kwargs.pkl',
)
path_params = hf_hub_download(
    repo_id='PierreGtch/EEGNetv4',
    filename='EEGNetv4_BNCI2014004/model-params.pkl',
)
with open(path_kwargs, 'rb') as f:
    kwargs = pickle.load(f)
module_cls = kwargs['module_cls']
module_kwargs = kwargs['module_kwargs']

# load the model with pre-trained weights:
model = module_cls(**module_kwargs)

/Users/sun/miniconda3/envs/biomed/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 